In [581]:
import cadquery as cq
from jupyter_cadquery import set_defaults, set_sidecar, open_viewer, PartGroup, Part
from jupyter_cadquery.viewer.client import show, show_object

from types import SimpleNamespace as d

from collections.abc import Iterable
import types, copy

class WPPart(Part):
    def __init__(self, wp, cq_obj, *args, **kwargs):
        self.wp = wp
        super().__init__(cq_obj, *args, **kwargs)

    def __getattr__(self, attr):
        return getattr(self.wp, attr)
    
    def __hasattr__(self, attr):
        if attr == 'objects': return False
        return hasattr(self.cq_shape, attr)

class ProxyMethodWrapper:
    def __init__(self, obj, func, name):
        self.obj, self.func, self.name = obj, func, name

    def __call__(self, *args, **kwds):
        return self.obj._method_call(self.name, self.func, *args, **kwds)
    
class WPPartGroup(PartGroup):
    def __init__(self, wp, cq_objs, *args, **kwargs):
        self.wp = wp
        super().__init__(cq_objs, *args, **kwargs)

    def __getattr__(self, attr):
        att = getattr(self.wp, attr)
        if type(att) is types.MethodType:
            return ProxyMethodWrapper(self, att, attr)
        else:
            return attr
    
    def __hasattr__(self, attr):
        return hasattr(self.wp, attr)
    
    def do(self, elems, name=None):
        res = self.wp.do(elems, name)
        self.objects.append(res)
        return self

    def do_cut(self, elems, name=None):
        res = self.wp.do_cut(elems, name)
        self.objects.append(res)
        return self

    def _method_call(self, name, func, *args, **kwargs):
        res = func(*args, **kwargs)
        self.wp = get_wp(func(*args, **kwargs))
        self.objects[-1].objects.append(Part(self.wp, name + ident(self.wp)))
        return self


def new_wp(self):
    def copy_attrs(fro, to, attrs):
        for att in attrs:
            setattr(to, att, copy.copy(getattr(fro, att)))
    
    wp = self.workplane()
    wp.parent = None
    wp.ctx = cq.cq.CQContext()
    copy_attrs(self.ctx, wp.ctx, ['pendingWires', 'pendingEdges', 'firstPoint', 'tolerance', 'tags'])
    wp._tag = None
    return wp

def get_wp(wp_like):
    return wp_like if isinstance(wp_like, cq.Workplane) else wp_like.wp

def ident(o):
    return '-' + str(id(o))[-5:]

def to_part(wp, val, **kwargs):
    def make_unpacked(fn, name):
        o = fn(new_wp(wp))
        if isinstance(o, PartGroup):
            if len(o.objects) == 1: return o.objects[0]
            if name: o.name = name
            return o
        elif isinstance(o, Part):
            if name: o.name = name
            return o
        else:
            kwargs['name'] = name
            return Part(o, **kwargs)

    if isinstance(val, Iterable):
        return make_unpacked(val[0], val[1])
    return make_unpacked(val, getattr(val, 'name') if hasattr(val, 'name') else (val.__class__.__name__ + ident(val)))

def _workplane_do(self, elems, name=None):
    #cq_wp = self.wp if hasattr(self, 'wp') else self
    cq_wp = get_wp(self)
    if name is None: name = 'Group' + ident(self)
    return WPPartGroup(cq_wp, [Part(self.findSolid(), name = name), PartGroup([to_part(cq_wp, elem) for elem in elems], name = name + ':do')])
    #return PartGroup([Part(self.findSolid(), name = name), PartGroup([to_part(cq_wp, elem) for elem in elems], name = name + ':do')])

def _workplane_do_cut(self, elems, name=None):
    def cut_all(wp, part):
        if isinstance(part, PartGroup):
            return wp.cut(part.compound())
        elif isinstance(part, Part):
            return cut_all(wp, part.cq_shape)
        elif isinstance(part, Iterable):
            for p in part:
                wp = wp.cut(p)
            return wp
        elif isinstance(part, cq.Workplane):
            return wp.cut(part.findSolid())
        else:
            return wp.cut(part)

    if name is None: name = 'Group' + ident(self)
    cq_wp = self if isinstance(self, cq.Workplane) else self.wp
    parts = [to_part(cq_wp, elem, color = '#707070', show_faces = False) for elem in elems]
    wp = cq_wp.findSolid()
    for p in parts:
        wp = cut_all(wp, p)
    return WPPartGroup(cq_wp.newObject([wp]), [Part(wp, name = name), PartGroup(parts, name + ':do_cut'), PartGroup([], 'workplane')])
    #return PartGroup([Part(wp, name = name), PartGroup(parts, name + ':do_cut')])

cq.Workplane.do = _workplane_do
cq.Workplane.do_cut = _workplane_do_cut

In [582]:
cq.Workplane().box(10, 10, 10).faces(">Z").do_cut([
    (lambda wp: wp.box(5, 5, 5), 'cut'),
    (lambda wp: wp.box(2, 2, 8), 'cut1')
]).faces(">Z").fillet(1).faces("<Z").chamfer(1).do([
    (lambda wp: wp.box(2, 2, 8), 'add2')
])

CadViewerWidget(anchor=None, cad_width=800, height=600, pinning=False, theme='light', title=None, tree_width=2…

In [292]:
def gt2_belt(l, w, oversize = 0.0):
    # veeery approximately a 2GT/GT2 belt.
    def make(wp):
        h, p, tooth_h, tooth_r = 1.38 + oversize, 2.0, 0.75 - oversize, 0.555 + 2 * oversize
        belt_h = h - tooth_h
        def make1(sketch, pos):
            x, y = pos
            x1 = x + (p - 2 * tooth_r) / 2
            x2 = x1 + 2 * tooth_r
            return (sketch
                    .segment( (x, y), (x, y + belt_h) ).segment( (x1, belt_h) ).segment( (x1, h - tooth_r) )
                    .arc( (x1, h - tooth_r), (x1 + tooth_r, h), (x2, h - tooth_r) )
                    .segment( (x2, belt_h) ).segment( (x + p, belt_h) ).segment( (x + p, y) ).close().assemble().finalize().extrude(w)
                   )
        profile = make1(wp.sketch(), (0, 0))
        res = profile
        for i in range(int(l / p) - 1):
            res = res.union(profile.translate( (p * (i + 1), 0, 0) ))
        return res
    return make

belt_d = d(w = 1.35 + 0.4, h = 6.0)

gt2_belt = gt2_belt(20, belt_d.h, oversize = 0.07)(cq.Workplane())

gt2_belt

CadViewerWidget(anchor=None, cad_width=800, height=600, pinning=False, theme='light', title=None, tree_width=2…

In [339]:
not_centered = { 'centered': (False, False, False) }

def make_points(base_point, extents, offsets):
    return map(lambda p: (base_point[0] + p[0] * extents[0], base_point[1] + p[1] * extents[1]), offsets)

not_centered = { 'centered': (False, False, False) }

def make_points(base_point, extents, offsets):
    return map(lambda p: (base_point[0] + p[0] * extents[0], base_point[1] + p[1] * extents[1]), offsets)

stepper_35_d = d(w = 36, h = 36, l = 22.0, bolt_dd = 26, bolt_inset = 4.25, bolt_D = 6.2, conn = d(h = 16.5, w = 10.0, l = 7.0))
mgn12h_d = d(hole_dx = 3.5, hole_D = 2.5 + 0.5, w = 27.0, l = 45.4, bolt_dd = 20.0)
mount_d = d(mot = stepper_35_d,
            l = 36.5,
            l_tot = 36.5,
            bolt_dl = 24.5,
            bolt_dd = 15.0,
            bolt_D = 2.6,
            th_top = 2.0,
            th_bot = 3.5,
            th = 2.0,
            mgn12_hole_inset = 1.0,
            fila_cog_l = 5.0,
            fila_cog_w = 20.0)
M2_5_screw_csk_d = d(D = 2.5 + 0.4, D_top = 5.0 + 0.2, csk_angle = 82)
M3_screw_csk_d = d(D = 3.0 * 1.2, D_top = 5.7 * 1.2, csk_angle = 82)
belt_d = d(w = 1.35 + 0.4, h = 6.0)
heat_sert_d = d(D_sm = 3.9, l = 3.0, D_lg = 5.0)

rail_top_to_carriage_bot = 16.5

belt_attachment_d = d(th = 3.5, l = 9.8, lower_dz = 15.0 - rail_top_to_carriage_bot, higher_dz = 7.0 - rail_top_to_carriage_bot, belt = belt_d)
belt_attachment_front_d = d(th = 3.5, l = 6.0, lower_dz = 15.0 - rail_top_to_carriage_bot, higher_dz = 7.0 - rail_top_to_carriage_bot, belt = belt_d)

In [583]:
def _workplane_centerBetween(self, direction="x"):
    if self.size() < 2: raise ValueError('Expecting two values on the stack')
    edges = getattr(self.item(0).val().Center(), direction), getattr(self.item(1).val().Center(), direction)
    newCenter = self.plane.origin
    print('ct1', newCenter)
    setattr(newCenter, direction, edges[1] + (edges[0] - edges[1]) / 2.0)
    print('ct2', newCenter)
    return self.newObject([newCenter])

cq.Workplane.centerBetween = _workplane_centerBetween

def belt_mount(belt_attachment_d, mount_d):
    def belt_mount_make(wp):
        b = belt_attachment_d
        m = mount_d
        w = m.mot.w + 2* m.th

        l_h, r_h = 12 - b.higher_dz, 20 - b.lower_dz
        b_w = 2 * b.th
        m_w = 2 * m.th

        a_z = 3 * b.belt.w + 2 * b.th
        belt_cut_z = 1.5 * b.belt.w

        lower_dz, higher_dz = b.lower_dz - b.belt.h / 2 - 2 * b.th, b.higher_dz - b.belt.h / 2 - 2 * b.th

        part = (
            wp.moveTo(0, m.th_bot).sketch()
                .segment( (0, -lower_dz), (0, -m.th_bot - b.l) ).segment( (-m.th, -m.th_bot - b.l) ).segment( (-m.th, -m.th_bot) )
                .segment( (-w + m.th, -m.th_bot) )
                .segment( (-w + m.th, -m.th_bot - b.l) ).segment( (-w, -m.th_bot - b.l) ).segment( (-w, -higher_dz) )
                .segment( (-w + b_w, -higher_dz) ).segment( (-w + b_w, 0) ).segment( (-b_w, 0) ).segment( (-b_w, -lower_dz))
                .close().assemble().finalize().extrude(a_z)
                .faces("<Y").edges(">Z").chamfer(min([a_z, b.l]))
                .faces(">Y or <Y[1]").edges("|Z").fillet(2)
                .faces(">Z").tag('Z1').do_cut([
                    ((lambda wp: wp.faces(tag='Z1').edges('<X or <<X[7]').centerBetween().move(0,-higher_dz - b.belt.h / 2).circle(heat_sert_d.D_sm / 2).extrude(-heat_sert_d.l)), 'belt_slot_r'),
                    ((lambda wp: wp.faces(tag='Z1').edges('>X or <<X[5]').centerBetween().move(0, -lower_dz - b.belt.h / 2).circle(heat_sert_d.D_sm / 2).extrude(-heat_sert_d.l)), 'belt_slot_l'),
                ])
                .do_cut([
                    ((lambda wp: wp.moveTo(-b.lower_dz, a_z / 2).rect(1.5 * b.belt.h, belt_cut_z).extrude(b_w)))#.workplane(offset = m.mot.w / 2).moveTo(-b.lower_dz, a_z / 2).rect(1.5 * b.belt.h, 2 * belt_cut_z).cutBlind(-2 * b.th)), 'belt_slot_r')
                ])
        )
        return part
    return belt_mount_make

belt_mount(belt_attachment_front_d, mount_d)(cq.Workplane())

ct1 Vector: (0.0, 0.0, 12.25)
ct2 Vector: (-36.5, 0.0, 12.25)
ct1 Vector: (0.0, 0.0, 12.25)
ct2 Vector: (-3.5, 0.0, 12.25)
     


CadViewerWidget(anchor=None, cad_width=800, height=600, pinning=False, theme='light', title=None, tree_width=2…

In [134]:
def mount(mount = mount_d, rail_mount_fasteners = M3_screw_csk_d, rail_top_to_carriage_bot = rail_top_to_carriage_bot, rail = mgn12h_d, belt = belt_d):
    m = mount
    w, h = m.mot.w + 2* m.th, m.mot.h + m.th_top + m.th_bot
    rail_mount_pts = make_points( (w / 2 - rail.bolt_dd / 2, -rail.hole_dx - m.mgn12_hole_inset), (rail.bolt_dd, rail.bolt_dd), [ (0, 0), (1, 0), (0, -1), (1, -1) ])        
    
    def boxx(wp):
        return wp.box(10, 10, 10)

    return  (
        cq.Workplane().box(w, h, m.l, **not_centered)
            .faces(">Z").workplane().moveTo(w/2, h/2 + ( m.th_top - m.th_bot)/2).rect(m.mot.w, m.mot.h).cutThruAll().edges("|Z").chamfer(2)
            .faces("<X").workplane().moveTo(-m.th_top - (m.mot.h - m.bolt_dd) / 2, -(m.l - m.bolt_dl)).cboreHole(m.bolt_D, 2 * m.bolt_D, m.th / 4, m.th + 1)
                                    .moveTo(-m.th_top - (m.mot.h - m.bolt_dd) / 2 - m.bolt_dd, -(m.l - m.bolt_dl)).cboreHole(m.bolt_D, 2 * m.bolt_D, m.th / 4, m.th + 1)
                                    # motor wire cut-out right:
                                    # .moveTo(-m.th_top - (m.mot.h) / 2, -(m.l - m.mot.conn.w / 2)).rect(m.mot.conn.h, m.mot.conn.w).cutBlind(-m.mot.conn.l)
            .faces(">X").workplane().moveTo( m.th_top + (m.mot.h - m.bolt_dd) / 2, -(m.l - m.bolt_dl)).cboreHole(m.bolt_D, 2 * m.bolt_D, m.th / 4, m.th + 1)
                                    .moveTo( m.th_top + (m.mot.h - m.bolt_dd) / 2 + m.bolt_dd, -(m.l - m.bolt_dl)).cboreHole(m.bolt_D, 2 * m.bolt_D, m.th / 4, m.th + 1)
                                    # motor wire cut-out left:
                                    .moveTo( m.th_top + (m.mot.h) / 2, -(m.l - m.mot.conn.w / 2)).rect(m.mot.conn.h, m.mot.conn.w).cutBlind(-m.mot.conn.l)
            .faces(">Y[1]").workplane().pushPoints(make_points( (w / 2 - rail.bolt_dd / 2, -rail.hole_dx - m.mgn12_hole_inset), (rail.bolt_dd, rail.bolt_dd), [ (0, 0), (1, 0), (0, -1), (1, -1) ])).circle(rail_mount_fasteners.D / 2 + 1.5).cutBlind(-m.th_top)
            .faces(">Y[4]").workplane().pushPoints( make_points( (- w / 2 - rail.bolt_dd / 2, -rail.hole_dx - m.mgn12_hole_inset), (rail.bolt_dd, rail.bolt_dd), [ (0, 0), (1, 0), (0, -1), (1, -1) ])).cskHole(rail_mount_fasteners.D, rail_mount_fasteners.D_top, rail_mount_fasteners.csk_angle, m.th_bot)
            .faces(">X").workplane()
                                    # fila advance cog cut-out:
                                    .moveTo( m.th_top -(m.mot.h) / 2, -m.fila_cog_l / 2).rect(m.fila_cog_w, m.fila_cog_l).cutBlind(-m.mot.conn.l)
            #.faces(">Z").do([
            #    (belt_mount(belt_attachment_front_d, mount_d), 'belt_mount_front')
            #], '>Z')
    )
    
mount()

CadViewerWidget(anchor=None, cad_width=800, height=600, pinning=False, theme='light', title=None, tree_width=2…

In [ ]:
import cadquery as cq
from jupyter_cadquery import set_defaults, set_sidecar

from types import SimpleNamespace as d
from jupyter_cadquery.viewer.client import show, show_object

not_centered = { 'centered': (False, False, False) }

def make_points(base_point, extents, offsets):
    return map(lambda p: (base_point[0] + p[0] * extents[0], base_point[1] + p[1] * extents[1]), offsets)

stepper_35_d = d(w = 36, h = 36, l = 22.0, bolt_dd = 26, bolt_inset = 4.25, bolt_D = 6.2, conn = d(h = 16.5, w = 10.0, l = 7.0))
mgn12h = d(hole_dx = 3.5, hole_D = 2.5 + 0.5, w = 27.0, l = 45.4, bolt_dd = 20.0)
mount_d = d(mot = stepper_35_d,
            l = 36.5,
            l_tot = 36.5,
            bolt_dl = 24.5,
            bolt_dd = 15.0,
            bolt_D = 2.6,
            th_top = 2.0,
            th_bot = 3.5,
            th = 2.0,
            mgn12_hold_inset = 1.0,
            fila_cog_l = 5.0,
            fila_cog_w = 20.0)
m2_5 = d(D = 2.5 + 0.4, D_top = 5.0 + 0.2, csk_angle = 82)
m3 = d(D = 3.0 * 1.2, D_top = 5.7 * 1.2, csk_angle = 82)
belt = d(w = 1.35 + 0.4, h = 6.0)

rail_top_to_carriage_bot = 16.5

m = mount_d

w, h = m.mot.w + 2* m.th, m.mot.h + m.th_top + m.th_bot
top_holes = make_points( (w / 2 - mgn12h.bolt_dd / 2, -mgn12h.hole_dx - m.mgn12_hold_inset), (mgn12h.bolt_dd, mgn12h.bolt_dd), [ (0, 0), (1, 0), (0, -1), (1, -1) ])

mount = (
    cq.Workplane().box(w, h, m.l, **not_centered)
        .faces(">Z").workplane().moveTo(w/2, h/2 + ( m.th_top - m.th_bot)/2).rect(m.mot.w, m.mot.h).cutThruAll().edges("|Z").chamfer(2)
        .faces("<X").workplane().moveTo(-m.th_top - (m.mot.h - m.bolt_dd) / 2, -(m.l - m.bolt_dl)).cboreHole(m.bolt_D, 2 * m.bolt_D, m.th / 4, m.th + 1)
                                .moveTo(-m.th_top - (m.mot.h - m.bolt_dd) / 2 - m.bolt_dd, -(m.l - m.bolt_dl)).cboreHole(m.bolt_D, 2 * m.bolt_D, m.th / 4, m.th + 1)
                                # motor wire cut-out right: 
                                # .moveTo(-m.th_top - (m.mot.h) / 2, -(m.l - m.mot.conn.w / 2)).rect(m.mot.conn.h, m.mot.conn.w).cutBlind(-m.mot.conn.l)
        .faces(">X").workplane().moveTo( m.th_top + (m.mot.h - m.bolt_dd) / 2, -(m.l - m.bolt_dl)).cboreHole(m.bolt_D, 2 * m.bolt_D, m.th / 4, m.th + 1)
                                .moveTo( m.th_top + (m.mot.h - m.bolt_dd) / 2 + m.bolt_dd, -(m.l - m.bolt_dl)).cboreHole(m.bolt_D, 2 * m.bolt_D, m.th / 4, m.th + 1)
                                # motor wire cut-out left: 
                                .moveTo( m.th_top + (m.mot.h) / 2, -(m.l - m.mot.conn.w / 2)).rect(m.mot.conn.h, m.mot.conn.w).cutBlind(-m.mot.conn.l)
        .faces(">Y[1]").workplane().pushPoints(make_points( (w / 2 - mgn12h.bolt_dd / 2, -mgn12h.hole_dx - m.mgn12_hold_inset), (mgn12h.bolt_dd, mgn12h.bolt_dd), [ (0, 0), (1, 0), (0, -1), (1, -1) ])).circle(m3.D / 2 + 1.5).cutBlind(-m.th_top)
        .faces(">Y[4]").workplane().pushPoints( make_points( (- w / 2 - mgn12h.bolt_dd / 2, -mgn12h.hole_dx - m.mgn12_hold_inset), (mgn12h.bolt_dd, mgn12h.bolt_dd), [ (0, 0), (1, 0), (0, -1), (1, -1) ])).cskHole(m3.D, m3.D_top, m3.csk_angle, m.th_bot)
        .faces(">X").workplane()
                                # fila advance cog cut-out:
                                .moveTo( m.th_top -(m.mot.h) / 2, -m.fila_cog_l / 2).rect(m.fila_cog_w, m.fila_cog_l).cutBlind(-m.mot.conn.l)
)

belt_attachment_d = d(th = 3.5, l = 9.8, lower_dz = 15.0 - rail_top_to_carriage_bot, higher_dz = 7.0 - rail_top_to_carriage_bot, belt = belt)
belt_attachment_front_d = d(th = 3.5, l = 6.0, lower_dz = 15.0 - rail_top_to_carriage_bot, higher_dz = 7.0 - rail_top_to_carriage_bot, belt = belt)

def belt_attachment(workplane, mount_d = mount_d, belt_attachment_d = belt_attachment_d, rail_d = d(), rotate = (1, 0, 0, 180)):    
    b = belt_attachment_d
    m = mount_d
    w = m.mot.w + 2* m.th
    
    l_h, r_h = 12 - b.higher_dz, 20 - b.lower_dz


    # belt_mounts.rear_dx is from rail (20mm wide), so convert to dy from ">Z". End of rail coincides with center of carriage M2.5 bolt holes (both 20mm wide).
    #b_dy = m.mgn12_hold_inset + mgn12h.bolt_dd + b.rear_dy - m.l - b.l / 2
    
    b_w = 2 * b.th
    m_w = 2 * m.th
    
    a_z = 3 * b.belt.w + 2 * b.th
    belt_cut_z = 1.5 * b.belt.w
    
    lower_dz, higher_dz = b.lower_dz - b.belt.h / 2 - 2 * b.th, b.higher_dz - b.belt.h / 2 - 2 * b.th 
    
    attachment = (
        workplane.workplane()
            .sketch().segment( (0, higher_dz), (0, m.th_bot + b.l) ).segment( (m.th, m.th_bot + b.l) ).segment( (m.th, m.th_bot) )
                     .segment( (w - m.th, m.th_bot) )
                     .segment( (w - m.th, m.th_bot + b.l) ).segment( (w, m.th_bot + b.l) ).segment( (w, lower_dz) )
                     .segment( (w - b_w, lower_dz) ).segment( (w - b_w, 0) ).segment( (b_w, 0) ).segment( (b_w, higher_dz))
                .close().assemble().finalize().extrude(a_z).mirror(mirrorPlane="YZ", basePointVector=(0, 0, 0))
    )
    attachment = attachment.rotateAboutCenter(cq.Vector(*rotate[:3]), rotate[-1])
    return attachment


def union_with_faces_xy(obj1, obj2, obj1_left_face, obj1_top_face, obj2_left_face, obj2_top_face, dz = 0):
    dx = obj1_left_face.Center().x - obj2_left_face.Center().x
    dy = obj1_top_face.Center().y - obj2_top_face.Center().y
    print(dx, dy, obj1_top_face.Center(), obj2_top_face.Center())
    obj2 = obj2.translate( (dx, dy, dz) )
    return obj1.union(obj2), obj2

top = mount.faces(">Y").val()
left = mount.faces(">X").val()

rear_belt = belt_attachment(cq.Workplane("XY"), rotate = (1, 0, 0, 180))
mount, _ = union_with_faces_xy(mount, rear_belt, top, left, rear_belt.faces(tag='left').val(), rear_belt.faces(tag='top').val(), dz = -8)

front_belt = belt_attachment(cq.Workplane("XY"), belt_attachment_d = belt_attachment_front_d, rotate = (0, 0, -1, 180))
#mount, res = union_with_faces_xy(mount, front_belt, left, top, front_belt.faces(tag='left').val(), front_belt.faces(tag='top').val(), dz = m.l)

# Cutout for filament lever.
hh = 20
#mount = mount.faces(">Y[-6]").workplane().moveTo(w -  hh / 2 - 2 * belt_attachment_d.th , 8).rect(hh, 16).cutBlind(-m.th_bot)

# Nozzle bottom cutout.
hh = 14
#mount = mount.faces("<Y[-1]").workplane().moveTo(- w / 2, -1).slot2D(hh, 10).cutBlind(-m.th_top)

# Heatsert holes for PCB holder.
heat_sert_d = d(D_sm = 3.9, l = 3.0, D_lg = 5.0)
hh = 12
y = -m.l - 2 * belt_attachment_d.belt.w
#mount = mount.faces(">Y[-6]").workplane().moveTo(w / 2 - hh / 2 - m.th, y).lineTo(w / 2  + hh / 2 - m.th, y).vertices().circle(heat_sert_d.D_sm / 2).cutBlind(-m.th_bot)

from jupyter_cadquery import PartGroup, Part

PartGroup([ Part(mount, "m"), Part(rear_belt, 'b'), Part(rear_belt.faces(tag='base_top'), 'bt'), Part(top, 't')])

In [274]:
from cadquery import exporters

exporters.export(mount, 'biqu_h2o_mg12h_mount_v2.stl')

In [329]:
import cadquery as cq
from jupyter_cadquery import set_defaults, set_sidecar

from types import SimpleNamespace as d
from jupyter_cadquery.viewer.client import show, show_object

not_centered = { 'centered': (False, False, False) }

# tringular_section_d = d(w = 13.3, l = 13.3, th_bot = 2.75, h_rect = 4, h = 11.45, hole_D = 3.2, hole_edge_dh = 6.31, hole_center_dh = 8.0, w_top = 7.8, th_side_r = 2.1, th_side_l = 1.5)
# lever_d = d(w = 4.6, l = 28.0, h = 8.5, hole_D = 4.7, hole_dl = 1.5)

# Adjusted to my printer/filament's dimensions.
tringular_section_d = d(w = 13.3, l = 13.3, th_bot = 2.75, h_rect = 4, h = 11.45, hole_D = 3.4, hole_edge_dh = 6.31, hole_center_dh = 8.0, w_top = 7.8, th_side_r = 2.1, th_side_l = 1.5)
lever_d = d(w = 5.1, l = 28.0, h = 8.5, hole_D = 4.95, hole_dl = 1.5)

t = tringular_section_d
l = lever_d

lever_triangular = (
    cq.Workplane("YZ")
                    .sketch()
                        .segment( (0, 0), (t.w, 0) ).segment( (t.w, t.h_rect) ).segment( (t.w / 2 + t.w_top / 2, t.hole_center_dh) ).arc((t.w / 2, t.hole_center_dh), t.w_top / 2, 0., 360).segment( (t.w / 2 - t.w_top / 2, t.hole_center_dh) ).segment( (0, t.h_rect) ).close().hull()
                        .push( [(t.w / 2, t.hole_center_dh)] ).circle(t.hole_D / 2, mode = 's').finalize()
                    .extrude(t.l)
                    .faces("<Y").workplane().moveTo((t.th_side_r - t.th_side_l) / 2 + t.w / 2, t.h/2 + t.th_bot).rect(t.w - t.th_side_r - t.th_side_l, t.h).cutThruAll()
)
face = lever_triangular.faces("<Y").tag('joint')
lever_triangular = lever_triangular.workplaneFromTagged('joint').moveTo(l.w/2, l.h/2).rect(l.w, l.h).workplane(offset=l.l).moveTo(l.w/2, l.h/2).rect(l.w, l.h/2).loft()
lever_triangular = lever_triangular.workplaneFromTagged('joint').moveTo(t.th_side_r/2, l.h/2).rect(t.th_side_r, l.h).extrude(-t.hole_center_dh/2).faces("<Y[1]").edges("<X").fillet(1.5)
lever_triangular = lever_triangular.workplaneFromTagged('joint').transformed(rotate=(0, 90, 0), offset=(0, l.h/2, l.hole_D/2 + l.hole_dl)).circle(l.hole_D/2).cutBlind(l.w)
lever_triangular

In [209]:
from cadquery import exporters

exporters.export(lever_triangular, 'biqu_h2o_lever_slim.stl')

In [ ]:
def old_belt_attachments(mount):
    return (
        mount
            .faces("<Z").workplane(offset = b_dy).moveTo(-b.th, b.higher_dz).box(b.th, l_h, b.l, **not_centered).moveTo(-w, b.lower_dz).box(b.th, r_h, b.l, **not_centered)
            .faces(">Z").workplane().moveTo(-w/2, -h/2 + m.th_top/2 + m.th_bot/2).sketch().rect(m.mot.w, m.mot.h).vertices().chamfer(2).finalize().cutThruAll()
    )

def belt_attachment_o(workplane, mount_d = mount_d, belt_attachment_d = belt_attachment_d, rail_d = d(), rotate = (1, 0, 0, 180)):    
    b = belt_attachment_d
    m = mount_d
    w = m.mot.w + 2* m.th
    
    l_h, r_h = 12 - b.higher_dz, 20 - b.lower_dz


    # belt_mounts.rear_dx is from rail (20mm wide), so convert to dy from ">Z". End of rail coincides with center of carriage M2.5 bolt holes (both 20mm wide).
    #b_dy = m.mgn12_hold_inset + mgn12h.bolt_dd + b.rear_dy - m.l - b.l / 2
    
    m_w = 2 * m.th
    
    a_z = 3 * b.belt.w + 2 * b.th
    belt_cut_z = 1.5 * b.belt.w
    
    lower_dz, higher_dz = b.lower_dz - b.belt.h / 2 - 2 * b.th, b.higher_dz - b.belt.h / 2 - 2 * b.th 
    
    attachment = (
        workplane.workplane()
            .sketch().segment( (0, higher_dz), (0, m.th_bot + b.l) ).segment( (m.th, m.th_bot + b.l) ).segment( (m.th, m.th_bot) )
                     .segment( (w - m.th, m.th_bot) )
                     .segment( (w - m.th, m.th_bot + b.l) ).segment( (w, m.th_bot + b.l) ).segment( (w, lower_dz) )
                     .segment( (w - b_w, lower_dz) ).segment( (w - b_w, 0) ).segment( (b_w, 0) ).segment( (b_w, higher_dz))
                .close().assemble().finalize().extrude(a_z)
                .faces("<X[-1] or <X[-3]").edges("<Y").fillet(b.th / 2).faces("<X[0] or <X[2]").edges("<Y").fillet(b.th / 2)
                .faces(">Y").edges(">Z").chamfer(3)
                .faces("<X[-1]").workplane().moveTo(-b.higher_dz, a_z / 2).rect(1.5 * b.belt.h, belt_cut_z).cutBlind(-b_w).workplane(offset = m.mot.w / 2 - b.th).moveTo(-b.higher_dz, a_z / 2).rect(1.5 * b.belt.h, 2 * belt_cut_z).cutBlind(-2 * b.th)
                .faces("<X[0]").workplane().moveTo(-b.lower_dz, a_z / 2).rect(1.5 * b.belt.h, belt_cut_z).cutBlind(-b_w).workplane(offset = m.mot.w / 2).moveTo(-b.lower_dz, a_z / 2).rect(1.5 * b.belt.h, 2 * belt_cut_z).cutBlind(-2 * b.th)
                .faces("<X[3]").edges(">Y").fillet(3)
                .faces("<X[-4]").edges(">Y").fillet(1)
                .mirror(mirrorPlane="YZ", basePointVector=(0, 0, 0))
    )
    attachment = attachment.rotateAboutCenter(cq.Vector(*rotate[:3]), rotate[-1])
    return attachment